In [ ]:
# The idea of this script is calculate size distrbutions of osm Features 
# of intresset is to calculate a distrbution for each key value pair that is 
# listed on the wikipages refrences in osm_groups.txt
# for this we will randomly querry the whole OSM dataset to collect samples

In [ ]:
# https://gis.stackexchange.com/questions/127427/transforming-shapely-polygon-and-multipolygon-objects

In [2]:
import parse_osm
from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, MultiPoint
import random
import pandas as pd

In [51]:
def random_bb(size,constrain=None):
    # size has to be in degrees since to BB for osm is in degrees
    # constrain has to be a shapely (multi-)polygon and can be used to limit the area
    
    # if a constrain exist we shrink the possibility space donw to the bounds of it 
    if constrain:
        minx, miny, maxx, maxy = constrain.bounds
    else:
        minx, miny, maxx, maxy = -180,-90,180,90
    
    # we generate first the lower left corner of the bounding box
    if constrain:
        # when an constrain exist we make shure the point is within it
        ll_x,ll_y =-9999999,-9999999
        while not constrain.contains(Point(ll_x, ll_y)):
            ll_x,ll_y = random.uniform(minx, maxx),random.uniform(miny, maxy)
    else:
        ll_x,ll_y = random.uniform(minx, maxx),random.uniform(miny, maxy)
        
    
     
    return Polygon([(ll_x,ll_y),(ll_x+size,ll_y),(ll_x+size,ll_y+size),(ll_x,ll_y+size)])
    
    
def generate_tag_list(df):
    tag_list = []
    for row in df.iterrows():
        tag_list.append(f"{row[1]['key']} {row[1]['value']}")
    
    return tag_list

def filter_tags(row):
    # this function looks if key,value are present in the tag_list 
    # if so it returns true
    r_value = False
    if row['tags']:
        for key in row['tags'].keys():
            if f"{key} {row['tags'][key]}" in tag_list:
                 r_value = True
    return r_value

In [24]:
q_poly = Polygon([(52.450425727741,13.286182880402),(52.458323725344,13.286182880402),(52.458323725344,13.299744129181),(52.450425727741,13.299744129181)])
r_bb = random_bb(0.005,q_poly)
list(r_bb.exterior.coords)

[(52.450584180213696, 13.294114789471116),
 (52.4555841802137, 13.294114789471116),
 (52.4555841802137, 13.299114789471117),
 (52.450584180213696, 13.299114789471117),
 (52.450584180213696, 13.294114789471116)]

In [27]:
osm_handle = parse_osm.disect_osm(parse_osm.json_from_osm(r_bb))
osm_handle.feature_df

,type,id,geometry,tags
0,node,21487262,{'point': POINT (13.308262 52.4671013)},None
1,node,26162586,{'point': POINT (13.2603946 52.4351448)},None
2,node,26162587,{'point': POINT (13.264948 52.4355797)},None
3,node,26162588,{'point': POINT (13.2679826 52.436945)},None
4,node,26162590,{'point': POINT (13.2707148 52.4382201)},None
...,...,...,...,...
5825,way,887655171,"[8252906140, 8252919721, 8252919718, 825291971...",{'amenity': 'university'}
5826,way,887666652,"[8252969973, 7554401435, 8252969974, 825296998...",{'amenity': 'university'}
5827,way,887666653,"[8252969979, 8252969976, 8252969981, 825296997...",{'amenity': 'university'}
5828,way,887668839,"[7554401093, 8252965697, 8252965703, 825296569...",{'amenity': 'university'}


In [54]:
osm_handle.feature_df[osm_handle.feature_df.apply(filter_tags, axis= 1)]      

,type,id,geometry,tags
4439,node,7006420495,{'point': POINT (13.2947512 52.4543169)},{'amenity': 'bench'}
4976,node,8363092328,{'point': POINT (13.2954337 52.4548785)},"{'artwork_type': 'sculpture', 'description': '..."
4977,node,8363092329,{'point': POINT (13.295406 52.4549061)},"{'artwork_type': 'sculpture', 'description': '..."
4983,relation,539138,"[{'type': 'way', 'ref': 53793024, 'role': 'out...","{'building': 'yes', 'source': 'yahoo', 'type':..."
4992,relation,11148577,"[{'type': 'way', 'ref': 149956948, 'role': 'ou...","{'alt_name': 'FU Berlin', 'amenity': 'universi..."
...,...,...,...,...
5824,way,887654045,"[8252906138, 8252906139, 7554401421, 755440142...",{'landuse': 'residential'}
5825,way,887655171,"[8252906140, 8252919721, 8252919718, 825291971...",{'amenity': 'university'}
5826,way,887666652,"[8252969973, 7554401435, 8252969974, 825296998...",{'amenity': 'university'}
5827,way,887666653,"[8252969979, 8252969976, 8252969981, 825296997...",{'amenity': 'university'}


In [58]:
tag_df =  pd.read_pickle('./kv_df_just_eng.pickle')
tag_list = generate_tag_list(tag_df)
tag_df[tag_df['key']=='landuse']

,key,value,en,en_text
363,landuse,commercial,https://en.wikipedia.org/wiki/Business_park,a business park or office park is an area of l...
366,landuse,residential,https://en.wikipedia.org/wiki/Residential_area,a residential area is a land used in which hou...
367,landuse,retail,https://en.wikipedia.org/wiki/Commercial_area,commercial areas in a city are areas districts...
368,landuse,allotments,https://en.wikipedia.org/wiki/Allotment_(garde...,an allotment garden british english often call...
369,landuse,farmland,https://en.wikipedia.org/wiki/Agricultural_land,agricultural land is typically land devoted to...
372,landuse,forest,https://en.wikipedia.org/wiki/Forest,a forest is an area of land dominated by tree...
373,landuse,meadow,https://en.wikipedia.org/wiki/Meadow,a meadow is an open habitat or field vegetated...
374,landuse,orchard,https://en.wikipedia.org/wiki/Orchard,an orchard is an intentional plantation of tre...
375,landuse,vineyard,https://en.wikipedia.org/wiki/Vineyard,a vineyard ˈvɪnjərd vin yərd also uk ˈvɪnjɑːrd...
377,landuse,brownfield,https://en.wikipedia.org/wiki/Brownfield_land,in urban planning brownfield land is any previ...


In [25]:
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, WKTLayer
from ipywidgets import Label

# osmid = test.feature_df[test.feature_df['type']=='relation'].sample().iloc[0]['id']
# test_multi = test.generate_geometry('relation',osmid)

m = Map(
    basemap=basemaps.CartoDB.Positron,
    # for some reason lat lon are switch for centering the map
    center=(r_bb.centroid.coords[0]),
    zoom=14
)


wlayer = WKTLayer(
    wkt_string=Polygon([(cord[1],cord[0]) for cord in r_bb.exterior.coords]).wkt,
    #hover_style={"fillColor": "red"},

)

label = Label()
display(label)



def handle_interaction(**kwargs):
    cords = '[]'
    if kwargs.get('type') == 'click':
        cords = f'{cords[:-1]}({kwargs.get("coordinates")[0]},{kwargs.get("coordinates")[1]})]'
        label.value = cords

m.on_interaction(handle_interaction)


m.add_layer(wlayer)
m

Label(value='')

Map(center=[52.4530841802137, 13.296614789471116], controls=(ZoomControl(options=['position', 'zoom_in_text', …